In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import re
import glob

In [ ]:
sns.set_style('ticks')
sns.set_context('paper')
sns.set_color_codes()

In [ ]:
def parse_output(f):
    times = []
    rates = []

    l = f.readline()
    m = re.search(r'Optimization.*O(\d)', l)
    if m:
        opt_level = int(m.group(1))

    for l in f:
        m = re.search(r'For N=(\d+)', l)
        if m:
            dim = int(m.group(1))
            continue

        if re.search('Timings', l):
            list_to_append = times
            value_name = 'time'
            continue
        elif re.search('Performance', l):
            list_to_append = rates
            value_name = 'rate'
            continue

        m = re.search('(Regular|Blocked):\s*(\d+\.\d+e[\+,\-]\d+)\s*(\d+\.\d+e[\+,\-]\d+)\s*(\d+\.\d+e[\+,\-]\d+)', l)
        if m:
            list_to_append.append({'N': dim, 'opt_level': opt_level, 'algo': m.group(1), value_name: float(m.group(3))})
        continue

    times = pd.DataFrame(times)
    rates = pd.DataFrame(rates)
    
    return pd.merge(times, rates, how='outer')

In [ ]:
res = []
for fp in glob.glob('outputs/*.o*'):
    with open(fp, 'r') as f:
        r = parse_output(f)
    res.append(r)
    
res = pd.concat(res, axis=0, ignore_index=True)

In [ ]:
fg = sns.FacetGrid(res, col='opt_level', col_wrap=2, size=2, hue='algo', hue_kws={"marker": ["^", "v"]}, palette='Set1')
fg.map(plt.plot, 'N', 'time')
fg.add_legend(title='Algorithm')
fg.set_titles("Opt. level O{col_name}")
fg.set_axis_labels('Dimension of matrix', 'Time [s]')
plt.xlim(0, 2100)
plt.savefig('writeup/opttime.pdf')

In [ ]:
fg = sns.FacetGrid(res, col='opt_level', col_wrap=2, hue='algo', size=2, hue_kws={"marker": ["^", "v"]}, palette='Set1')
fg.map(plt.plot, 'N', 'rate')
fg.add_legend(title='Algorithm')
fg.set_titles("Opt. level O{col_name}")
fg.set_axis_labels('Dimension of matrix', 'Rate [FLOPS]')
plt.xlim(0, 2100)
plt.savefig('writeup/optrate.pdf')

In [ ]:
fg = sns.FacetGrid(res, hue='opt_level', col='algo', size=2, hue_kws={"marker": ["^", "v", 's', 'o']}, palette='Set1')
fg.map(plt.plot, 'N', 'time')
fg.add_legend(title='Opt. level')
fg.set_titles("{col_name}")
fg.set_axis_labels('Dimension of matrix', 'Time [s]')
plt.xlim(0, 2100)
plt.savefig('writeup/algotime.pdf')

In [ ]:
fg = sns.FacetGrid(res, hue='opt_level', col='algo', size=2, hue_kws={"marker": ["^", "v", 's', 'o']}, palette='Set1')
fg.map(plt.plot, 'N', 'rate')
fg.add_legend(title='Opt. level')
fg.set_titles("{col_name}")
fg.set_axis_labels('Dimension of matrix', 'Rate [FLOPS]')
plt.xlim(0, 2100)
plt.savefig('writeup/algorate.pdf')

In [ ]:
res.to_csv('res.csv')